In [91]:
# # import torch
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# # # or any of these variants
# # # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# # # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# # # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# # # model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
# # model.eval()

# from PIL import Image
# from torchvision import transforms

# filename = 'dog.jpg'

# input_image = Image.open(filename)
# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# input_tensor = preprocess(input_image)
# input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# # move the input and model to GPU for speed if available
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     model.to('cuda')

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# # print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# # print(probabilities)

# # Read the categories
# with open("imagenet_classes.txt", "r") as f:
#     categories = [s.strip() for s in f.readlines()]
# # Show top categories per image
# top5_prob, top5_catid = torch.topk(probabilities, 5)
# for i in range(top5_prob.size(0)):
#     print(categories[top5_catid[i]], top5_prob[i].item())


In [1]:
import torch
import torch.nn as nn

learning_rate = 1e-3 #@param
batch_size = 64 #@param
epochs = 2000 #@param

NUM_CLASSES = 22
LABELS = [
    "ape", "bear", "bison", "cat",
    "chicken", "cow", "deer", "dog",
    "dolphin", "duck", "eagle", "fish",
    "horse", "lion", "lobster", "pig",
    "rabbit", "shark", "snake", "spider",
    "turkey", "wolf"
]
LABEL_MAP = {
    0: "ape", 1: "bear", 2: "bison", 3: "cat",
    4: "chicken", 5: "cow", 6: "deer", 7: "dog",
    8: "dolphin", 9: "duck", 10: "eagle", 11: "fish",
    12: "horse", 13: "lion", 14: "lobster",
    15: "pig", 16: "rabbit", 17: "shark", 18: "snake",
    19: "spider", 20:  "turkey", 21: "wolf"
}
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [93]:
import os
from torchvision.datasets import VisionDataset
from PIL import Image
import torchvision.models as models
import torchvision

class TEST(VisionDataset):
    def __init__(self,root, transform, target_transform):
        self.img_dir = root
        self.transform = transform
        self.target_transform = target_transform
        self.samples = os.listdir(self.img_dir)

    def __len__(self):
        
        return len(self.samples)

    def __getitem__(self, index):
        img_path = os.path.normpath(os.path.join(self.img_dir, self.samples[index]))
        with open(img_path, "rb") as f:
            img = Image.open(f).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

from torchvision.datasets import ImageFolder, VisionDataset
from torchvision import transforms
from torch.utils.data import DataLoader
data_path = 'Animals/Animals_Dataset' #@param
batch_size = 16 #@param
num_workers = 2 #@param

train_path = os.path.normpath((os.path.join(data_path, 'train')))
test_path = os.path.normpath((os.path.join(data_path, 'test')))

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

label2onehot = transforms.Lambda(lambda y: torch.zeros(
    NUM_CLASSES, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

train_dataset = ImageFolder(
    train_path,
    preprocess,
    target_transform = label2onehot
)
test_dataset = TEST(
    test_path,
    transform = preprocess,
    target_transform = label2onehot 
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    num_workers=num_workers) #!TODO: rewrite this statement
model = models.resnet18(weights = torchvision.models.ResNet18_Weights.DEFAULT)

In [94]:
import torch.nn as nn
fc_infeatures = model.fc.in_features
model.fc = nn.Linear(fc_infeatures, NUM_CLASSES)
model.to(device)
loss_fn = nn.CrossEntropyLoss()

In [95]:
class AverageMeter(object):
  """Computes and stores the average and current value"""
  def __init__(self, name, fmt=':f'):
      self.name = name
      self.fmt = fmt
      self.reset()

  def reset(self):
      self.val = 0
      self.avg = 0
      self.sum = 0
      self.count = 0

  def update(self, val, n=1):
      self.val = val
      self.sum += val * n
      self.count += n
      self.avg = self.sum / self.count

  def __str__(self):
      fmtstr = '{name} {avg' + self.fmt + '}'
      return fmtstr.format(**self.__dict__)

def accuracy(output:torch.Tensor, target, topk=(1,)):
  """Computes the accuracy over the k top predictions for the specified values of k"""
  with torch.no_grad():
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    print(pred.shape, target.shape)
    pred = pred.t()
    print(target.view(1, -1).shape)
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
losses = AverageMeter('Loss', ':.4e')
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')

In [96]:
fc_params_id = list(map(id, model.fc.parameters()))     
base_params = filter(lambda p: id(p) not in fc_params_id, model.parameters())
optimizer = torch.optim.SGD([
    {'params': base_params, 'lr': learning_rate*0},   #冻结卷积层的方法，
    {'params': model.fc.parameters(), 'lr': learning_rate}], momentum=0.9)

In [97]:
import time
start = time.time()
for i in range(epochs):
  for batch, (X, y) in enumerate(train_loader):
    X = X.to(device)
    y = y.to(device)
    data_time.update(time.time() - start)
    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # acc1, acc5 = accuracy(pred, y, topk=(1, 5))
    losses.update(loss.item(), X.size(0))
    # top1.update(acc1[0], X.size(0))
    # top5.update(acc5[0], X.size(0))

  batch_time.update(time.time() - start)
  start = time.time()

  print(f"Epoch:{i + 1}: {batch_time}, {losses}")

PicklingError: Can't pickle <function <lambda> at 0x000001BABD1FB560>: attribute lookup <lambda> on __main__ failed

In [ ]:
torch.save(model, 'resnet.path')

In [ ]:
model = torch.load('resnet.path')

In [ ]:
model.eval()
labels = []
with torch.no_grad():
  for _, images in enumerate(test_loader):
    y = model(images.to(device))
    batch_labels = torch.argmax(y, dim=1)
    labels.append(batch_labels)
ans = torch.cat(labels, 0).cpu().numpy()
print(ans)
print([LABEL_MAP[i] for i in ans])